# Inline holography with a simulated dataset
This notebook showcases the functionality of the inline_holo python module using a simulated focal series dataset. These functionalities are accessed using the scripts below, that can be taken as example code, modified and reused in different applications. The simulated dataset is generated from an input wave (Wave.hdf5) and parameters, also given below together in a separate cell. Multi-focus TIE and gaussian-process TIE algorithms are used to obtain the low-spatial frequency information from the focal series. The Gerchberg-Saxton algorithm is finally used to also include high-spatial frequencies in these solutions. Validation tests are produced to explore the quality of these solutions.

In [1]:
# some useful tools
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np

import hyperspy.api as hs

# inline_holo tools
from image_tools import *
from inline_holo import *
from inline_holo import ModifiedImage as MI
from inline_holo import ComplexModifiedImage as CMI

from time import time as t

The cell below defines some scripts used to run the calculations and to store the results in a consistent manner.

In [2]:
use_gpu = False

In [3]:
def noiser(fs, SNR, seed=None):
    # Remove padding and add some noise
    fsn = fs.unset_padding()
    fsn.data += add_noise_gauss(fsn.data, SNR,seed)

    # Re-do padding
    fsu = fsn.set_padding(((Nypad, Nypad), (Nxpad, Nxpad)), 'reflect')
    #fsu = fsn.set_padding(((200, 200), (200, 200)), 'constant', constant_values=fsn.data.mean())

    fsu.data = fsu.data.astype('float32')
    fsu.data = np.ascontiguousarray(fsu.data)
    fsu.data[fsu.data<0.] = 1e-4
    return fsu

def three_plane_slicer(fs, idz):
    # idz = 1, 2, 3...
    Ndz = np.int(np.floor(len(fs) / 2))
    n1 = Ndz-idz
    n2 = idz-Ndz if Ndz != idz else None
    fs_idz=fs.inav[n1:n2:idz]
    fs_idz.axes_manager[0].axis = zdef[n1:n2:idz]
    return fs_idz

def do_mftie(fsi, SNR, lothres, hithres, Nbutter=100):
    results = {'SNR': SNR, 
               'hithres': hithres, 
               'lothres': lothres,
               'defocus' : fsi.axes_manager[0].axis}

    # MFTIE
    mftie = MFTIE(fsi, use_gpu, lothres, hithres, alpha, SNR, Nbutter=Nbutter)
    phase = mftie(False)
    
    # MFTIE, validate
    PhV = phase.unset_padding().set_padding(((Nypad, Nypad), (Nxpad, Nxpad)), 'reflect')
    val = validation(PhV, phase_org, False)
    frc = val.run_fourier_ring_correlation(d2d=False)
    rmse_real, rmse_rcpr = val.run_rmse_check() 
    
    # save MFTIE results
    inv_lap = fsi.inav[len(fsi)//2+1:].deepcopy()
    if use_gpu:
        inv_lap.data = mftie.inverse_laplacian.get()
        results.update({'Phi_dz': mftie.phdata.get()})
    else:
        inv_lap.data = mftie.inverse_laplacian.copy()
        results.update({'Phi_dz': mftie.phdata.copy()})
        
    results.update({'Phi' : phase})
    results.update({'Nm2' : inv_lap})
    results.update({'k2H' : mftie.k2_hi_thres})
    results.update({'k2L' : mftie.k2_lo_thres})
    results.update({'frc' : frc})
    results.update({'RMSE2' : mftie.rmse2n})
    
    rmse_rcpr.data = np.fft.fftshift(rmse_rcpr.data, (-2,-1))
    results.update({'rms' : rmse_rcpr.integrate_radial(512)})
    
    return results

def do_gptie(fsi, Nsl):
    results = {'Nsl': Nsl, 
               'defocus' : fsi.axes_manager[0].axis[-1]}

    # GPTIE
    gptie = GPTIE(fsi, Nsl=Nsl)
    phase = gptie(False)
    
    # GPTIE, validate
    PhV = phase.unset_padding().set_padding(((Nypad, Nypad), (Nxpad, Nxpad)), 'reflect')
    val = validation(PhV, phase_org, False)
    frc = val.run_fourier_ring_correlation(d2d=False)
    rmse_real, rmse_rcpr = val.run_rmse_check() 
    
    
    # save GPTIE results
    results.update({'Phi' : phase})
    results.update({'frc' : frc})
    
    rmse_rcpr.data = np.fft.fftshift(rmse_rcpr.data, (-2,-1))
    results.update({'rms' : rmse_rcpr.integrate_radial(512)})
    
    return results

def do_gsref(fsi, wave_init, ctf, Niters):
    results = {'Nsl': Niters, 
               'defocus' : fsi.axes_manager[0].axis[-1]}

    # GSref
    gsref = GS(fsi, wave_init, ctf, alpha, use_gpu)
    wave = gsref(Niters)
    
    # GSref, validate phase
    phase = wave.deepcopy()
    phase.data = np.angle(wave.data) 
    PhV = phase.unset_padding().set_padding(((Nypad, Nypad), (Nxpad, Nxpad)), 'reflect')
    val = validation(PhV, phase_org, False)
    frc_phi = val.run_fourier_ring_correlation(d2d=False)
    rmse_real, rmse_rcpr_phi = val.run_rmse_check() 
    
    # GSref, validate intensity
    amplitude = wave.deepcopy()
    amplitude.data = np.abs(wave.data) 
    AmV = phase.unset_padding().set_padding(((Nypad, Nypad), (Nxpad, Nxpad)), 'reflect')
    val = validation(AmV, ampli_org, False)
    frc_amp = val.run_fourier_ring_correlation(d2d=False)
    rmse_real, rmse_rcpr_amp = val.run_rmse_check() 
    
    # save GSref results
    results.update({'Phi' : phase})
    results.update({'frc_phi' : frc_phi})
    
    results.update({'Amp' : amplitude})
    results.update({'frc_amp' : frc_amp})
    
    rmse_rcpr_phi.data = np.fft.fftshift(rmse_rcpr_phi.data, (-2,-1))
    results.update({'rms_phi' : rmse_rcpr_phi.integrate_radial(512)})
    
    rmse_rcpr_amp.data = np.fft.fftshift(rmse_rcpr_amp.data, (-2,-1))
    results.update({'rms_amp' : rmse_rcpr_amp.integrate_radial(512)})
    
    return results

## 1. Define input parameters

In [4]:
# PARAMETERS
alpha       = 0.       # uScope: Convergergence (set to 0 for coherent illumiation)
NA          = 0.6      # uScope: Num Aperture 
wlen        = 0.5      # uScope: Wave-length
nref        = 1.       # uScope: Refractive index

defmin      = 1.0      # Experiment: Smallest defocus plane
defmax      = 150.     # Experiment: Largest defocus plane
Ndef        = 10       # Experiment: Number of defocus planes

fsmooth     = 2.       # FS-Simulation: Cosine-Bell smooth 

lothres     = 1.0      # MFTIE: low-freq threshold
hithres     = 0.25     # MFTIE: high-freq threshold
Nsl         = 50       # GPTIE: binning
Niters      = 25       # GSref: Iterations

In [5]:
# load original wave
wave = hs.load('Wave.hdf5')
wave.__class__ = CMI
Ny, Nx = wave.unset_padding().axes_manager.signal_shape
Nypad, Nxpad = (int(Ny/2), int(Nx/2))
wave = wave.unset_padding().set_padding(((Nypad, Nypad), (Nxpad, Nxpad)), 'reflect')

# calculate original phase
phase_org = wave.deepcopy()
phase_org.data = np.angle(wave.data)

# calculate original amplitude
ampli_org = wave.deepcopy()
ampli_org.data = np.abs(wave.data)

In [6]:
# Useful to know
wnum = 2. * np.pi / wlen
q = np.linspace(0., wnum, 256)
q2= q**2.

This below is a plot of the input wave

In [7]:
wave.unset_padding().plot(representation='polar')

## 2.A: 3 defoci experiment

Select the defocus planes for the simulation: 1, 5 and 10

In [8]:
SNR = 20

In [9]:
dz = np.array([1., 5., 10.])
zdef = np.concatenate( [-dz[::-1], np.array([0,]), dz])
print(np.round(zdef,1))

[-10.  -5.  -1.   0.   1.   5.  10.]


In [10]:
from CTFSim import CTFSim

# Project the wave to the selected planes
t0 = t()
ctsim = CTFSim(wave, zdef, alpha, use_gpu, None, wlen, nref, NA, 2.)
fs = ctsim(False)
Tsim = t()-t0
print('CTF-Sim time:', Tsim)

del CTFSim, ctsim 

C:\Users\aeljarrat\inline_holo\inline_holo.py:538: RuntimeWarning: invalid value encountered in sqrt
  Chi = np.sqrt(wnum**2-k2) - wnum


CTF-Sim time: 3.1626360416412354


In [11]:
# Apply noise
fsn = noiser(fs, SNR, seed=12345)

# Do TIE !!!
sim_mfivx = do_mftie(fsn, SNR, lothres, hithres, Nbutter=1000)

# Get in-focus Amplitude
Nm  = get_middle(fsn)
amp = fsn.inav[Nm]

# Get init-waves
wave_ivx = sim_mfivx['Phi'].deepcopy()
wave_ivx.data = np.sqrt(amp.data) * np.exp(1j*sim_mfivx['Phi'].data)

# Get CTF
ctf = amp.get_contrast_transfer(zdef)

# Do GSref !!
sim_gsivx = do_gsref(fsn, wave_ivx, ctf, Niters)

C:\Users\aeljarrat\inline_holo\MFTIE.py:294: RuntimeWarning: divide by zero encountered in true_divide
  nabla_minus_2 = (1. / k2) * (1./(1.+(k2/k2Hi[:,None,None])**Nbutter)) * \
C:\Users\aeljarrat\inline_holo\MFTIE.py:294: RuntimeWarning: overflow encountered in power
  nabla_minus_2 = (1. / k2) * (1./(1.+(k2/k2Hi[:,None,None])**Nbutter)) * \
C:\Users\aeljarrat\inline_holo\MFTIE.py:295: RuntimeWarning: overflow encountered in power
  (1.-1./(1.+(k2/k2Lo[:,None,None])**Nbutter))
C:\Users\aeljarrat\inline_holo\MFTIE.py:295: RuntimeWarning: invalid value encountered in multiply
  (1.-1./(1.+(k2/k2Lo[:,None,None])**Nbutter))
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 12.01it/s]
C:\Users\aeljarrat\inline_holo\inline_holo.py:538: RuntimeWarning: invalid value encountered in sqrt
  Chi = np.sqrt(wnum**2-k2) - wnum
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 10.78

## 2.B: 21 defoci experiment

Select the defocus planes for the simulation: geometric progression. 

In [12]:
dz = np.logspace(np.log10(defmin), np.log10(defmax), 10)
zdef = np.concatenate( [-dz[::-1], np.array([0,]), dz])
print(np.round(zdef,1))

[-150.   -86.   -49.3  -28.2  -16.2   -9.3   -5.3   -3.    -1.7   -1.
    0.     1.     1.7    3.     5.3    9.3   16.2   28.2   49.3   86.
  150. ]


Additional noise level is higher.

In [13]:
SNR = 15

In [14]:
from CTFSim import CTFSim

# Project the wave to the selected planes
t0 = t()
ctsim = CTFSim(wave, zdef, alpha, use_gpu, None, wlen, nref, NA, 2.)
fs = ctsim(False)
Tsim = t()-t0
print('CTF-Sim time:', Tsim)

del CTFSim, ctsim 

C:\Users\aeljarrat\inline_holo\inline_holo.py:538: RuntimeWarning: invalid value encountered in sqrt
  Chi = np.sqrt(wnum**2-k2) - wnum


CTF-Sim time: 9.042755842208862


In [15]:
# Apply noise
fsn = noiser(fs, SNR, seed=12345)

# Do TIE !!!
sim_mftie = do_mftie(fsn, SNR, lothres, hithres, Nbutter=10)
sim_gptie = do_gptie(fsn, Nsl)

# Get in-focus Amplitude
Nm  = get_middle(fsn)
amp = fsn.inav[Nm]

# Get init-waves
wave_mf = sim_mftie['Phi'].deepcopy()
wave_mf.data = np.sqrt(amp.data) * np.exp(1j*sim_mftie['Phi'].data)

wave_gp = sim_gptie['Phi'].deepcopy()
wave_gp.data = np.sqrt(amp.data) * np.exp(1j*sim_gptie['Phi'].data)

# Get CTF
ctf = amp.get_contrast_transfer(zdef)

# Do GSref !!
sim_mf_gs = do_gsref(fsn, wave_mf, ctf, Niters)
sim_gp_gs = do_gsref(fsn, wave_gp, ctf, Niters)

C:\Users\aeljarrat\inline_holo\MFTIE.py:294: RuntimeWarning: divide by zero encountered in true_divide
  nabla_minus_2 = (1. / k2) * (1./(1.+(k2/k2Hi[:,None,None])**Nbutter)) * \
C:\Users\aeljarrat\inline_holo\MFTIE.py:295: RuntimeWarning: invalid value encountered in multiply
  (1.-1./(1.+(k2/k2Lo[:,None,None])**Nbutter))
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 11.93it/s]
C:\Users\aeljarrat\inline_holo\GPTIE.py:149: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  Coeff = np.linalg.lstsq(L, np.linalg.lstsq(L.T, D)[0])[0]
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 11.85it/s]
C:\Users\aeljarrat\inline_holo\inline_holo.py:53

## 3. plotting time

In [16]:
psi_list = [sim_gsivx['Phi'].unset_padding() + 0.3, sim_gsivx['Amp'].unset_padding(),  
            sim_mf_gs['Phi'].unset_padding() + 0.3, sim_mf_gs['Amp'].unset_padding(),   
            sim_gp_gs['Phi'].unset_padding() + 0.3, sim_gp_gs['Amp'].unset_padding()]

frc_list = [[sim_mfivx['frc'], sim_gsivx['frc_phi']],
            [sim_mftie['frc'], sim_mf_gs['frc_phi']],
            [sim_gptie['frc'], sim_gp_gs['frc_phi']]]

rms_list = [[sim_mfivx['rms'], sim_gsivx['rms_phi']],
            [sim_mftie['rms'], sim_mf_gs['rms_phi']],
            [sim_gptie['rms'], sim_gp_gs['rms_phi']]]

kax = sim_mftie['frc'].axes_manager[-1].axis

dz = sim_mfivx['defocus'].copy()
dz = dz[len(dz)//2+1:]
x_arrstr = np.char.mod('%i', dz)

dz = sim_mftie['defocus'].copy()
dzM = round(dz[len(dz)//2+1:].max(),1)
dzm = round(dz[len(dz)//2+1:].min(),1)
labels = [r'MFTIE, $\delta z='+",".join(x_arrstr)+'\mu m$',
          r'MFTIE, $\delta z='+np.str(dzm)+'-'+np.str(dzM)+'\mu m$',
          r'GPTIE, $\delta z='+np.str(dzm)+'-'+np.str(dzM)+'\mu m$',] 


### Plot reconstructed waves 
These are obtained after TIE inversion and GS refinement.

In [17]:
from string import ascii_lowercase as alph

size_factor = 2

h, w = plt.figaspect(3/2.)
h, w = size_factor*h, size_factor*w
f = plt.figure(figsize=(h, w))

#f = plt.figure()
axs = hs.plot.plot_images(psi_list, per_row=2, scalebar=[0,], colorbar=None, axes_decor='off', fig=f)

# set amplitude clim
for io, axi in enumerate(axs[::2]):
    imi = axi.images[0]
    imi.set_clim(vmin=0., vmax=1.)
    axi.set_title('('+alph[2*io]+')', loc='left', fontweight='bold')
    axi.text(0.02, 0.97, r'$\Phi_0$', transform=axi.transAxes, 
             fontweight='bold', color='white', ha='left', va='top', alpha=0.5)
    
# set phases clim
for io, axi in enumerate(axs[1::2]):
    imi = axi.images[0]
    imi.set_clim(vmin=0.8, vmax=3.2)
    axi.set_title('('+alph[2*io+1]+')', loc='left', fontweight='bold')
    axi.text(0.98, 0.97, labels[io][:5]+'+'+str(Niters)+'xGS\n'+labels[io][6:], 
             transform=axi.transAxes, 
             fontweight='bold', color='white', ha='right', va='top', alpha=0.5)
    axi.text(0.02, 0.97, r'$I_0$', transform=axi.transAxes, 
             fontweight='bold', color='white', ha='left', va='top', alpha=0.5)
    
sbar = axs[0].scalebar
sbar.line.set_lw(4.)
oldtxt = sbar.text.get_text()
sbar.text.set_text(oldtxt[:6]+'${\mu m}$')

f.show()
plt.pause(0.1) # wtf matplotlib

new_pos = (sbar.position[0]+10, sbar.position[1]-5.)
sbar.text.set_position(new_pos)

# custom scalebars, horizontal
posA = axs[-2].get_position()
caxA = f.add_axes([posA.x0, posA.y0-0.03, posA.width, 0.02])
cbarA = f.colorbar(axs[-2].images[0], orientation='horizontal', cax=caxA)
cbarA.set_ticks([0.0, 0.5, 1.0])

posB = axs[-1].get_position()
caxB = f.add_axes([posB.x0, posB.y0-0.03, posB.width, 0.02])
cbarB = f.colorbar(axs[-1].images[0], orientation='horizontal', cax=caxB)
cbarB.set_ticks([1., 2., 3.])

#f.subplots_adjust(wspace=0.1, hspace=0.15)

## Plot reconstruction validations

### Fourier ring correlation (FRC)

In [18]:
from string import ascii_lowercase as alph

Nplots = len(frc_list)
fig, axs = plt.subplots(Nplots, 1, sharex=True, figsize=(11.,8.))

for io, axi in enumerate(axs):
    axi.plot(kax, frc_list[io][0], '--k')
    axi.plot(kax, frc_list[io][1], '-k')
    axi.grid(which='major',axis='y')
    axi.set_yticks([0., 0.25, 0.5, 0.75, 1.])
    axi.set_ylim(0., 1.05)
    axi.set_xlim(0., wnum)
    axi.set_title(labels[io], loc='right')
    axi.set_title('('+alph[io]+')', loc='left', fontweight='bold')
    #axi.set_xscale('log')

axi.legend(['FRC-$\Phi$, TIE', 'FRC-$\Phi$, TIE+GS'], loc='lower left', frameon=False)
axs[-1].set_xlabel(r'$q=|\mathbf{q}|$, spatial frequency modulus $[{\mu m}^{-1}]$')
md=Nplots // 2
axs[md].set_ylabel(r'FRC-$\Phi$, Fourier ring correlation, phase [a.u.]') 

Text(0,0.5,'FRC-$\\Phi$, Fourier ring correlation, phase [a.u.]')

### Root-mean-square error (RMSE)

In [23]:
from string import ascii_lowercase as alph

Nplots = len(rms_list)
fig, axs = plt.subplots(Nplots,1, sharex=True, figsize=(11.,8.))

for io, axi in enumerate(axs):
    s1 = rms_list[io][0]
    kax1 = s1.axes_manager.signal_axes[0].axis
    axi.plot(kax1, s1.data, '-k')
    
    s2 = rms_list[io][1]
    kax2 = s2.axes_manager.signal_axes[0].axis
    axi.plot(kax2, s2.data, '--k')
    axi.grid(which='major',axis='y')
    
    axi.set_yscale('log')
    axi.set_xscale('log')
    axi.set_ylim(1e0, 1e4)
    axi.set_xlim(0., wnum)
    
    axi.set_title(labels[io], loc='right')
    axi.set_title('('+alph[io]+')', loc='left', fontweight='bold')
    
    
axi.legend(['Error, TIE', 'Error, TIE+GS'], loc='upper right', frameon=False)

axs[-1].set_xlabel(r'$q=|\mathbf{q}|$, spatial frequency modulus $[{\mu m}^{-1}]$')
md=Nplots // 2
axs[md].set_ylabel(r'$d_q$RMSE$^2$, differential root-mean-square error  $[rad^2 {\mu m}]$')    

C:\Users\aeljarrat\AppData\Local\conda\conda\envs\hyperspy_dev\lib\site-packages\matplotlib\axes\_base.py:3129: UserWarning: Attempted to set non-positive xlimits for log-scale axis; invalid limits will be ignored.
  'Attempted to set non-positive xlimits for log-scale axis; '


Text(0,0.5,'$d_q$RMSE$^2$, differential root-mean-square error  $[rad^2 {\\mu m}]$')

## Some comparisons

In [35]:
from string import ascii_lowercase as alph

c1 = 'b'
c2 = 'r'

Nplots = len(frc_list)
f, ax = plt.subplots(Nplots,1, sharex=True, figsize=(11.,8.))
for io, axi in enumerate(ax):
    
    axi.plot(kax, frc_list[io][0], '-'+c1, label='FRC, TIE')
    axi.plot(kax, np.abs(frc_list[io][0]), '-'+c1, alpha=0.5)
    
    axi.plot(kax, frc_list[io][1], '--'+c1, label='FRC, TIE+GS')
    axi.plot(kax, np.abs(frc_list[io][1]), '--'+c1, alpha=0.5)
    
    axi.set_title(labels[io], loc='right')
    axi.set_title('('+alph[io]+')', loc='left', fontweight='bold')
    
    axi.set_ylim(0., 1.05)
    axi.tick_params('y', colors=c1)
    
    # twin axis
    axt = axi.twinx()
    s1 = rms_list[io][0]
    kax1 = s1.axes_manager.signal_axes[0].axis
    axt.plot(kax1, s1.data, '-'+c2, label=r'$d_q$RMSE$^2$, TIE' )
    s2 = rms_list[io][1]
    kax2 = s2.axes_manager.signal_axes[0].axis
    axt.plot(kax2, s2.data, '--'+c2, label=r'$d_q$RMSE$^2$, TIE+GS')
    
    axt.set_yscale('log')
    axt.set_xscale('log')
    axt.set_ylim(1e0, 1e4)
    axt.set_xlim(0., wnum)
    axt.tick_params('y', colors=c2)
        
f.axes[0].legend(loc='lower left', frameon=False)
f.axes[4].legend(loc='lower left', frameon=False)

ax[-1].set_xlabel(r'$q=|\mathbf{q}|$, spatial frequency modulus $[{\mu m}^{-1}]$')
md=Nplots // 2
ax[md].set_ylabel(r'FRC, Fourier ring correlation [a.u.]', color=c1)
f.axes[md+Nplots].set_ylabel(r'$d_q$RMSE$^2$, differential root-mean-square error  $[rad^2 {\mu m}]$', 
                             color=c2, va='bottom', rotation=270)    

C:\Users\aeljarrat\AppData\Local\conda\conda\envs\hyperspy_dev\lib\site-packages\matplotlib\axes\_base.py:3129: UserWarning: Attempted to set non-positive xlimits for log-scale axis; invalid limits will be ignored.
  'Attempted to set non-positive xlimits for log-scale axis; '


Text(0,0.5,'$d_q$RMSE$^2$, differential root-mean-square error  $[rad^2 {\\mu m}]$')

In [34]:
from string import ascii_lowercase as alph

c1 = 'b'
c2 = 'r'

Nplots = len(frc_list)
gs_kw = {'width_ratios':[1, 3], 'wspace':0.25}
f, ax = plt.subplots(Nplots,2, sharex='col', gridspec_kw=gs_kw, figsize=(11.,8.))

#kax = sim_chi_3p[0]['frc'].axes_manager[0].axis

# images
axs = ax[:,0]
phases = [sim_gsivx['Phi'].unset_padding() + 0.3,   
          sim_mf_gs['Phi'].unset_padding() + 0.3,    
          sim_gp_gs['Phi'].unset_padding() + 0.3]

for io, axi in enumerate(axs):
    imi = axi.imshow(phases[io].data)
    imi.set_clim(vmin=0., vmax=1.)
    axi.set_axis_off()
    #axi.set_title(r'$\delta z='+str(np.round(dz[io],0))+'\mu m$', loc='right')
    axi.set_title('('+alph[io]+')', loc='left', fontweight='bold')

# sbar tech
hs.hyperspy.drawing.widgets.ScaleBar(axi, '${\mu m}$', pixel_size=0.2)

# custom scalebar, horizontal
pos = axs[-1].get_position()
cax = f.add_axes([pos.x0, pos.y0-0.03, pos.width, 0.02])
cbar = f.colorbar(axs[-1].images[0], orientation='horizontal', cax=cax)
cbar.set_ticks([0.0, 0.5, 1.0])


# validations
axl = ax[:,1]
axr = []
for io, axi in enumerate(ax[:,1]):
    
    axi.plot(kax, frc_list[io][0], '-'+c1, label='TIE')
    axi.plot(kax, np.abs(frc_list[io][0]), '-'+c1, alpha=0.5)
    
    axi.plot(kax, frc_list[io][1], '--'+c1, label='TIE+GS')
    axi.plot(kax, np.abs(frc_list[io][1]), '--'+c1, alpha=0.5)
    
    axi.set_title(labels[io], loc='right')
    #axi.set_title('('+alph[io+3]+')', loc='left', fontweight='bold')
    
    axi.set_ylim(0., 1.05)
    axi.tick_params('y', colors=c1)
    
    # twin axis
    axt = axi.twinx()
    s1 = rms_list[io][0]
    kax1 = s1.axes_manager.signal_axes[0].axis
    axt.plot(kax1, s1.data, '-'+c2, label=r'$d_q$RMSE$^2$, TIE' )
    s2 = rms_list[io][1]
    kax2 = s2.axes_manager.signal_axes[0].axis
    axt.plot(kax2, s2.data, '--'+c2, label=r'$d_q$RMSE$^2$, TIE+GS')
    
    axt.set_yscale('log')
    axt.set_xscale('log')
    axt.set_ylim(1e0, 1e4)
    axt.set_xlim(0., wnum)
    axt.tick_params('y', which='both', colors=c2)
    axr.append(axt)
    
axl[-1].legend(loc='lower left', frameon=False)

ax[-1,1].set_xlabel(r'$q=|\mathbf{q}|$, spatial frequency modulus $[{\mu m}^{-1}]$')
md=Nplots // 2
ax[md,1].set_ylabel(r'FRC, Fourier ring correlation [a.u.]', color=c1)
f.axes[md+2*Nplots+1].set_ylabel(r'$d_q$RMSE$^2$, differential root-mean-square error  $[rad^2 {\mu m}]$', 
                               color=c2, va='bottom', rotation=270)  

C:\Users\aeljarrat\AppData\Local\conda\conda\envs\hyperspy_dev\lib\site-packages\matplotlib\axes\_base.py:3129: UserWarning: Attempted to set non-positive xlimits for log-scale axis; invalid limits will be ignored.
  'Attempted to set non-positive xlimits for log-scale axis; '


Text(0,0.5,'$d_q$RMSE$^2$, differential root-mean-square error  $[rad^2 {\\mu m}]$')

## Inverse Laplacian
This plot compares the multi-focus inverse Laplacian filters from MFTIE with the actual inverse Laplacian $q^{-2}$. The additional application of these filters has to be equal to this function. **This only holds if alpha is set to 0**, in case partial spatial coherence is present, the inverse Laplacian filters take this into account and the normalization is different than $q^{-2}$.

In [36]:
nm2 = sim_mftie['Nm2'].deepcopy()
nm2.data = np.fft.fftshift(nm2.data, axes = (-2, -1))

In [39]:
k2 = fsn.get_fourier_space()
kscale = np.sqrt(k2[1,0])
k2 = np.fft.fftshift(k2, (-2,-1))

# collect inverse Laplacian
iLap = np.concatenate( [nm2.data, 
                        np.sqrt((nm2.data**2.).sum(0))[None,...],
                        (1./k2)[None,...]] )

iLap = MI(iLap)

# compute integral
Ndec = 0
S_iLap_dr = iLap.integrate_radial()
# Do not plot this signal directly: 
# the scale is no longer linear for Ndec > 1!!!

C:\Users\aeljarrat\AppData\Local\conda\conda\envs\hyperspy_dev\lib\site-packages\ipykernel\__main__.py:8: RuntimeWarning: divide by zero encountered in true_divide

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 34.46it/s]

In [40]:
f = plt.figure()
ax = f.add_subplot(111)

qrnd = S_iLap_dr.axes_manager[-1].axis * kscale
for si in S_iLap_dr:
    ax.plot(qrnd, si.data, mew=0.5)

ax.set_yscale('log')
ax.set_xscale('log')
ax.set_ylim(1e-3, 1e3)
ax.set_xlim(0.03, wnum)
ax.set_xlabel(r'$q=|\mathbf{q}|$, spatial frequency modulus $[{\mu m}^{-1}]$')
ax.set_ylabel(r'$\hat\nabla^{-2}$, inverse Laplacian filters $[{\mu m}^2]$')    

Text(0,0.5,'$\\hat\\nabla^{-2}$, inverse Laplacian filters $[{\\mu m}^2]$')

# GFTIE experiment
This is sort of a different experiment. We remove the boundary conditions from the dataset and build them up again using the gradient-flipping algorithm (GFTIE). **This algorithm does not take into account partial spatial coherence**, meaning the experiment works better with alpha = 0.

In [41]:
from GFTIE import GFTIE

In [43]:
Npad = 100

fs_gftie = fsn.unset_padding()
fs_gftie = fs_gftie.set_padding(((Npad, Npad), (Npad, Npad)), 'constant', constant_values=fs_gftie.data.mean())

# Generate initial phase result
sim_gftie = do_mftie(fs_gftie, SNR, lothres, hithres, Nbutter=10)
ph_gftie = sim_gftie['Phi']

# Do GFTIE
D = GFTIE(fs_gftie, ph_gftie, 20, 5)
mftie = MFTIE(fs_gftie, False, 1e5, 0.25, alpha, SNR, True)
mftie.set_dzI(D)
ph_flip = mftie(False)

C:\Users\aeljarrat\inline_holo\MFTIE.py:294: RuntimeWarning: divide by zero encountered in true_divide
  nabla_minus_2 = (1. / k2) * (1./(1.+(k2/k2Hi[:,None,None])**Nbutter)) * \
C:\Users\aeljarrat\inline_holo\MFTIE.py:295: RuntimeWarning: invalid value encountered in multiply
  (1.-1./(1.+(k2/k2Lo[:,None,None])**Nbutter))


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 11.24it/s]C:\Users\aeljarrat\inline_holo\GFTIE.py:86: RuntimeWarning: divide by zero encountered in true_divide
  nabla_minus_2 = (1. / np.repeat(k2[None,...], len(adef), 0) )


Starting GF; Rlp = 5 um




  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

  5%|████▏                                                                              | 1/20 [00:02<00:38,  2.01s/it]

Chi2 = 0.0010280546603986054; dChi2 = 0.9998971945339601
:  10%|██▍                     | 2/20 [00:07<00:53,  2.99s/it]

Chi2 = 0.0005447173733054711; dChi2 = 0.47014745977196487
:  15%|███▍                   | 3/20 [00:12<01:02,  3.69s/it]

Chi2 = 0.000480639225496952; dChi2 = 0.11763558672578782
:  20%|████▊                   | 4/20 [00:17<01:06,  4.18s/it]

Chi2 = 0.00047932498946417573; dChi2 = 0.002734350346494125
:  25%|█████▎               | 5/20 [00:23<01:07,  4.53s/it]

Chi2 = 0.00047645446821027656; dChi2 = 0.005988674316997443
:  30%|██████▎              | 6/20 [00:28<01:06,  4.78s/it]

Chi2 = 0.00047771261287327; dChi2 = 0.002640639865797628
:  35%|████████▍               | 7/20 [00:33<01:04,  4.92s/it]

Chi2 = 0.0004783826911571906; 

GS refinement of the MFTIE and GFTIE results

In [44]:
# Get in-focus Amplitude
Nm  = get_middle(fs_gftie)
amp = fs_gftie.inav[Nm]

# Get init-waves
wave_gf = sim_gftie['Phi'].deepcopy()
wave_gf.data = np.sqrt(amp.data) * np.exp(1j*ph_flip.data)

wave_mf = sim_gftie['Phi'].deepcopy()
wave_mf.data = np.sqrt(amp.data) * np.exp(1j*sim_gftie['Phi'].data)

# Get CTF
ctf = amp.get_contrast_transfer(zdef)

# Do GSref !!
sim_gf_gs = do_gsref(fs_gftie, wave_gf, ctf, Niters)
sim_mf_gs = do_gsref(fs_gftie, wave_mf, ctf, Niters)

C:\Users\aeljarrat\inline_holo\inline_holo.py:538: RuntimeWarning: invalid value encountered in sqrt
  Chi = np.sqrt(wnum**2-k2) - wnum



  0%|                                                                                           | 0/25 [00:00<?, ?it/s]


  4%|███▎                                                                               | 1/25 [00:11<04:35, 11.48s/it]


  8%|██████▋                                                                            | 2/25 [00:22<04:23, 11.45s/it]


 12%|█████████▉                                                                         | 3/25 [00:34<04:14, 11.55s/it]


 16%|█████████████▎                                                                     | 4/25 [00:46<04:03, 11.58s/it]


 20%|████████████████▌                                                                  | 5/25 [00:58<03:52, 11.62s/it]


 24%|███████████████████▉                                                               | 6/25 [01:09<03:40, 11.63s/it]


 28%|██

Plot the results

In [57]:
phase_org = MI(phase_org.real)

In [60]:
plt_list = [phase_org, ph_gftie, ph_flip, 
            phase_org, sim_mf_gs['Phi'], sim_gf_gs['Phi']]
lbl_list = [r'$\Phi$', 'MFTIE', 'GFTIE',
            r'$\Phi$', 'MFTIE+GS', 'GFTIE+GS']

In [61]:
f = plt.figure(figsize=(15., 15.))
hs.plot.plot_images(plt_list, label=lbl_list,
                    per_row=3, axes_decor='off', colorbar='single', fig=f)
plt.suptitle('gradient flipping results including pad region')

Text(0.5,0.98,'gradient flipping results including pad region')

In [62]:
f = plt.figure(figsize=(15., 15.))
hs.plot.plot_images([pi.unset_padding() for pi in plt_list], label=lbl_list,
                    per_row=3, axes_decor='off', colorbar='single', fig=f)
plt.suptitle('gradient flipping results including pad region')

Text(0.5,0.98,'gradient flipping results including pad region')

In [63]:
f, ax = plt.subplots(1,1, figsize=(11., 8.))
hs.plot.plot_spectra([sim_mf_gs['frc_phi'], sim_gf_gs['frc_phi']], 
                     legend=['MFTIE+GS', 'GFTIE+GS'], legend_loc='best', 
                     fig=f, ax=ax)
ax.set_ylim(0.25, 1.05)
ax.set_xscale('log')